# Empirically estimating p-value

> Using sklearn.model_selection.permutation_test_score, compute the p-value indicating if the score obtained by an instance of sklearn.dummy.DummyClassifier on the dataset sklearn.datasets.load_iris is obtained by chance.

P-value is about accepting or rejecting the following "null hypothesis", 

$ H_0 $ : there are no dependency between the target $ y $ and predictors $ X $

In simple terms, it is the probability that an estimator obtained its score on a dataset by chance and not by acquiring insight from features.

Let's take a look at the documentation of `sklearn.model_selection.permutation_test_score`.

> Permutes targets to generate ‘randomized data’ and compute the empirical p-value against the null hypothesis that features and targets are independent.

> The p-value represents the fraction of randomized data sets where the estimator performed as well or better than in the original data. A small p-value suggests that there is a real dependency between features and targets which has been used by the estimator to give good predictions. A large p-value may be due to lack of real dependency between features and targets or the estimator was not able to use the dependency to give good predictions.

This function provides a way to empirically compute the p-value associated to an estimator fitting a dataset `(X, y)`, by evaluating it on `n_permutations` shuffled datasets where targets are permuted. 

If the null hypothesis is false, then we expect our estimator to achieve a lower score on the vast majority of these randomized datasets.

In [1]:
from sklearn.datasets import load_iris

X, y = load_iris(return_X_y=True)


from sklearn.dummy import DummyClassifier

dummy_clf = DummyClassifier()


from sklearn.model_selection import permutation_test_score

score_dummy, permutation_scores_dummy, pvalue_dummy = permutation_test_score(dummy_clf, X, y, random_state=0)

print(f"P-value: {pvalue_dummy:.3f}")

print(f"Original Score: {score_dummy:.3f}")

print(f"Permutation Scores: {permutation_scores_dummy.mean():.3f} +/- {permutation_scores_dummy.std():.3f}")


P-value: 1.000
Original Score: 0.333
Permutation Scores: 0.333 +/- 0.000


With a p-value of `1.0` (the maximum possible), there are no doubt that the DummyClassifier gets its score by chance.

It means that for each permutation done, it achieved a final score greater or equal than the original score. 

Indeed, in the `prior` default strategy of that estimator, each prediction is simply the class label having the most occurrences, hence the same $ \hat{y} $ predictions vector, highlighted by `permutation_scores_dummy.std() == 0.0`.

If we try another strategy that does not always return the same class label like `uniform`, we should observe slight variations.

In [2]:
score_dummy_uniform, permutation_scores_dummy_uniform, pvalue_dummy_uniform = permutation_test_score(DummyClassifier(strategy='uniform'), X, y, random_state=0)

print(f"P-value: {pvalue_dummy_uniform:.3f}")

print(f"Original Score: {score_dummy_uniform:.3f}")

print(f"Permutation Scores: {permutation_scores_dummy_uniform.mean():.3f} +/- {permutation_scores_dummy_uniform.std():.3f}")

P-value: 0.772
Original Score: 0.313
Permutation Scores: 0.339 +/- 0.038


Anyway, as we usually accept the null hypothesis when $ p-value \geq 0.05 $, we are way above this thresold, and should definitely accept it.

But stricly speaking, it only tells us we accept the null hypothesis **for that estimator**. 
Like the documentation says, a high p-value indicates:

- no real dependency between features and targets 
- **OR** the estimator was not able to use the dependency to give good predictions


> Repeat this analysis using sklearn.ensemble.HistGradientBoostingClassifier instead of sklearn.dummy.DummyClassifier.
> What can you conclude about:
> - the existence of a significant statistical association between the iris type and the input features (petal and sepal width and length)?
> - the ability of each kind of estimator to assess or not such a statistical association between features and target variable?


In [3]:
from sklearn.ensemble import HistGradientBoostingClassifier

hist_clf = HistGradientBoostingClassifier().fit(X, y)

score_hist, permutation_scores_hist, pvalue_hist = permutation_test_score(hist_clf, X, y, random_state=0)


print(f"P-value: {pvalue_hist:.3f}")

print(f"Original Score: {score_hist:.3f}")

print(f"Permutation Scores: {permutation_scores_hist.mean():.3f} +/- {permutation_scores_hist.std():.3f}")
 

P-value: 0.010
Original Score: 0.947
Permutation Scores: 0.333 +/- 0.039


This time, the tested estimator achieves a much better score `0.947` on the original dataset, and drops to the `DummyClassifier` performance on randomized datasets.

The resulting p-value is thus very low, and we can safely reject the null hypothesis:
- the iris dataset contains real dependency between features and labels
- **AND** the classifier being able to use features to obtain good predictions

Going back to the `DummyClassifier` case, we can now then assert that it was not able to effectively learn on provided observations. 

That is actually what we expect from dummy models, which are nonetheless very useful to act as relevant **baselines** to rank other estimators.

This method is a simple way to explain what are p-values and their major importance when analyzing some data.

Moreover, it is applicable to any estimator and does not require any knowledge on labels/features distributions, but can quickly become heavy to run if we decide to keep a number of permutations in the same order of magnitude than the number of observations.

In [4]:
# execution infos

import sys
import sklearn

print("Python version")
print(sys.version)

print(f"{sklearn.__version__=}.")


Python version
3.9.12 (main, Apr  5 2022, 06:56:58) 
[GCC 7.5.0]
sklearn.__version__='1.3.2'.


: 